In [12]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
from db_utils import query_analytics_store


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


- [QuickSurveysResponses_15266417](https://meta.wikimedia.org/wiki/Schema:QuickSurveysResponses)
- [QuickSurveyInitiation_15278946](https://meta.wikimedia.org/wiki/Schema:QuickSurveyInitiation)
- [Survey](https://docs.google.com/spreadsheets/d/1JD8-knLmnFXVwXxJYx6w9RRmxZWasSLaSKvj85SgrzE/edit?ts=56c3ccd2#gid=1291145097)

In [13]:
response_table = 'log.QuickSurveysResponses_15266417'
impression_table = 'log.QuickSurveyInitiation_15278946'
host = 'analytics-store.eqiad.wmnet'

### EL Summary Stats

In [14]:
query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='eligible'
"""

n_eligible = query_analytics_store(query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='impression'
"""

n_impressions = query_analytics_store(query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
"""
n_responses = query_analytics_store(query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
WHERE
    event_surveyResponseValue ='ext-quicksurveys-external-survey-yes-button'
"""
n_yes = query_analytics_store( query, {}).iloc[0]['n']

query = """
SELECT
    COUNT(*) as n
FROM log.QuickSurveysResponses_15266417
WHERE
    event_surveyResponseValue ='ext-quicksurveys-external-survey-no-button'
"""
n_no = query_analytics_store( query, {}).iloc[0]['n']



print('Eligible Pageviews: ', n_eligible)
print('Widget Impressions: ', n_impressions)
print('Clicks: ', n_responses)
print('Yeses: ', n_yes)
print('Nos: ', n_no)

Eligible Pageviews:  44647
Widget Impressions:  34904
Clicks:  396
Yeses:  114
Nos:  282


### Does each eligible pageview have a unique survey token?

In [15]:
query = """
SELECT COUNT(DISTINCT(event_surveyInstanceToken)) as n
FROM log.QuickSurveyInitiation_15278946
WHERE event_eventName ='eligible'
"""
print(query_analytics_store(query, {}).iloc[0]['n'])

44641


Pretty Close

### Does each widget impression have a unique survey token?

In [16]:
query = """
SELECT COUNT(DISTINCT(event_surveyInstanceToken)) as n
FROM log.QuickSurveyInitiation_15278946
WHERE event_eventName ='impression'
"""
print(query_analytics_store(query, {}).iloc[0]['n'])

34890


Pretty Close.

### Does each Widget Impression have a matching Pageview?

In [17]:
query = """
SELECT
    COUNT(DISTINCT(i.event_surveyInstanceToken)) as n
FROM
    (SELECT event_surveyInstanceToken
    FROM log.QuickSurveyInitiation_15278946
    WHERE
    event_eventName ='impression') i
JOIN
    (SELECT event_surveyInstanceToken
    FROM log.QuickSurveyInitiation_15278946
    WHERE
    event_eventName ='eligible') e
ON
    (i.event_surveyInstanceToken = e.event_surveyInstanceToken)
"""

df = query_analytics_store(query, {})
print(df.iloc[0]['n'])

34831


Close. It seems there are a small number of impressions without a matching pageview.

### Does each click have a matching Widget Impression?

In [18]:
query = """
SELECT *
FROM
    log.QuickSurveysResponses_15266417 r,
    log.QuickSurveyInitiation_15278946 i
WHERE
    r.event_surveyInstanceToken = i.event_surveyInstanceToken
    AND i.event_eventName ='impression'
""" 

d_click = query_analytics_store( query, {})
print(d_click.shape[0], len(set(d_click['event_surveyInstanceToken'])))

393 393


Each click has exactly one widget impression. One click without a matching rendered widget

### Does each Google Survey Response have a registered click

In [20]:
d_survey = pd.read_csv('responses.tsv', sep = '\t')
st = 'This is you survey ID. Please do not modify.'
ct = 'event_surveyInstanceToken'
df_survey_click = d_survey.merge(d_click, how = 'inner', right_on = ct, left_on = st)

In [21]:
print(df_survey_click.shape[0])

59


In [23]:
print(d_survey.shape[0])

81


Only 59 out of 81 survey responses have a matching click.

### Does each Google Survey Response have a registered impression?

In [24]:
query = """
SELECT
    event_surveyInstanceToken
FROM log.QuickSurveyInitiation_15278946
WHERE
    event_eventName ='impression'
"""

d_impression = query_analytics_store(query, {})
df_survey_impression = d_survey.merge(d_impression, how = 'inner', right_on = 'event_surveyInstanceToken', left_on = st)

In [25]:
print(df_survey_impression.shape[0])

61


Only 61 out of 81 survey responses have a matching click.

### Summary

In [26]:
print('Eligible Pageviews: ', n_eligible)
print('Widget Impressions: ', n_impressions)
print('Clicks: ', n_responses)
print('Yeses: ', n_yes)
print('Google Responses tracked in EL', df_survey_click.shape[0])
print('Google Responses', d_survey.shape[0])

Eligible Pageviews:  44647
Widget Impressions:  34904
Clicks:  396
Yeses:  114
Google Responses tracked in EL 59
Google Responses 81


- Don't know how many users where in the sample
- One in 300 pageviews results in a 'Yes'
- 62% of 'Yes' events lead to a survery response
- only 60% of survey responses are trackable in EL

### Digging Into Missing Data

In [27]:
df_survey_click2 = d_survey.merge(d_click, how = 'left', right_on = ct, left_on = st)

In [28]:
d_survey.columns

Index(['Timestamp', 'I am reading this article to',
       'Prior to visiting this article', 'I am reading this article because',
       'This is you survey ID. Please do not modify.'],
      dtype='object')

In [29]:
df_survey_click2 = df_survey_click2[[st,'Timestamp', ct, 'timestamp']].sort('Timestamp')
df_survey_click2.columns = ['Google Token', 'Google Timestamp', 'EL Token', 'EL Timestamp']
df_survey_click2

,Google Token,Google Timestamp,EL Token,EL Timestamp
0,80c690e7c9f8b978,2/16/2016 16:18:23,80c690e7c9f8b978,20160217001810
1,bde401cf1103db40,2/16/2016 17:38:51,bde401cf1103db40,20160217013758
2,afa0344c0ede6223,2/16/2016 18:36:57,NaN,NaN
3,9da0dde789db8849,2/16/2016 18:48:20,NaN,NaN
4,da4e80310a906a6b,2/16/2016 19:18:12,da4e80310a906a6b,20160217031757
5,9b28ed09b46edefd,2/16/2016 19:36:57,NaN,NaN
6,6a1a72ec4545391c,2/16/2016 20:45:31,NaN,NaN
7,d46936d2a424b39c,2/16/2016 21:38:41,NaN,NaN
8,f9c14b602f76c17c,2/16/2016 21:38:43,f9c14b602f76c17c,20160217053758
9,943a5aa00b6cc088,2/17/2016 0:23:09,943a5aa00b6cc088,20160217082213


In [122]:
df_survey_click2['Google Token'].apply(len)

0     16
1     16
2     16
3     16
4     16
5     16
6     16
7     16
8     16
9     16
15    16
16    16
17    16
18    16
19    16
20    16
21    16
22    16
23    16
24    16
10    16
11    16
12    16
13    16
14    16
25    16
32    16
26    16
27    16
28    16
29    16
30    16
31    16
Name: Google Token, dtype: int64

No obvious pattern...

### Are surveySessionToken tokens unique per IP

In [30]:
query = """
SELECT
    COUNT(DISTINCT(clientIP)) as n
FROM log.QuickSurveyInitiation_15278946
"""
nIPs = query_analytics_store( query, {}).iloc[0]['n']
print(nIPs)

14850


In [31]:
query = """
SELECT
    COUNT(DISTINCT(CONCAT(clientIP, userAgent))) as n
FROM log.QuickSurveyInitiation_15278946
"""
nIPUAs = query_analytics_store( query, {}).iloc[0]['n']
print(nIPUAs)

15132


In [32]:
query = """
SELECT
    COUNT(DISTINCT(event_surveySessionToken)) as n
FROM log.QuickSurveyInitiation_15278946
"""
sessiontokens = query_analytics_store( query, {}).iloc[0]['n']
print(sessiontokens)

17786
